# Modules and data

In [1]:
from sqlalchemy import create_engine
import pandas as pd
import os

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/JacopoMalatesta/imdb_most_popular_films/main/data/cleaned_df.csv',
            index_col = 0)

# Secret info

Let's safely load our username and password with the OS module

In [3]:
username = os.environ.get('rds_mysql_imdb_username')

password = os.environ.get('rds_mysql_imdb_psw')

# Putting the data on the cloud

I created a MYSQL database on Amazon RDS by following this tutorial: https://aws.amazon.com/getting-started/hands-on/create-mysql-db/

Let's split the dataframe into three. 

In [4]:
films = df[['id', 'title', 'release_date', 'country', 'language', 'genre', 'studios', 'color', 'aspect_ratio', 'last_updated']]

stats = df[['id', 'runtime', 'budget', 'revenue', 'imdb_rating', 'imdb_rating_count', 'metascore', 'user_review_count',
           'critic_review_count']]

people = df[['id', 'director', 'writer', 'actors', 'cinematographer', 'editor', 'composer', 'production_designer', 
             'art_director','costume_designer', 'producers']]

Let's create the engine 

In [5]:
host = 'mysql-imdb.cxqopnqcl1bm.us-east-2.rds.amazonaws.com'
dbname = 'mysql_imdb'

connection = create_engine(f'mysql://{username}:{password}@{host}/{dbname}').connect()

Let's upload our three dataframes

In [6]:
films.to_sql(con = connection, name = 'films', if_exists = 'replace', chunksize = 500, index = False)
stats.to_sql(con = connection, name = 'stats', if_exists = 'replace', chunksize = 500, index = False)
people.to_sql(con = connection, name = 'people', if_exists = 'replace', chunksize = 500, index = False)

Let's run a simple to query to see if everything looks ok

In [7]:
pd.read_sql_query(sql = "SELECT * FROM people LIMIT 5", con = connection)

,id,director,writer,actors,cinematographer,editor,composer,production_designer,art_director,costume_designer,producers
0,tt0111161,Frank Darabont,Frank Darabont;Stephen King,Tim Robbins;Morgan Freeman;Bob Gunton;William ...,Roger Deakins,Richard Francis-Bruce,Thomas Newman,Terence Marsh;Soheil,Peter Landsdown Smith,Elizabeth McBride,Liz Glotzer;David V. Lester;Niki Marvin
1,tt0468569,Christopher Nolan,David S. Goyer;Christopher Nolan;Jonathan Nolan,Christian Bale;Heath Ledger;Aaron Eckhart;Mich...,Wally Pfister,Lee Smith,James Newton Howard;Hans Zimmer,Nathan Crowley,Mark Bartholomew;James Hambidge;Craig Jackson;...,Lindy Hemming,Kevin de la Noy;Jordan Goldberg;Philip Lee;Ben...
2,tt1375666,Christopher Nolan,Christopher Nolan,Leonardo DiCaprio;Joseph Gordon-Levitt;Elliot ...,Wally Pfister,Lee Smith,Hans Zimmer,Guy Hendrix Dyas,Luke Freeborn;Matthew Gray;Brad Ricker;Dean Wo...,Jeffrey Kurland,Zakaria Alaoui;John Bernard;Chris Brigham;Jord...
3,tt0137523,David Fincher,Chuck Palahniuk;Jim Uhls,Edward Norton;Brad Pitt;Meat Loaf;Zach Grenier...,Jeff Cronenweth,James Haygood,Dust Brothers;John King;Michael Simpson,Alex McDowell,Melique Berger;Chris Gorak,Michael Kaplan,Ross Grayson Bell;Ceán Chaffin;John S. Dorsey;...
4,tt0109830,Robert Zemeckis,Eric Roth;Winston Groom,Tom Hanks;Rebecca Williams;Sally Field;Michael...,Don Burgess,Arthur Schmidt,Alan Silvestri,Rick Carter,Leslie McDonald;William James Teegarden,Joanna Johnston,Wendy Finerman;Charles Newirth;Steve Starkey;S...
